# Analyze Telco Customer Churn: Data Prep
In preparation for a presentation to Telco executives about customer churn, the CFO is asking for an analysis and predictions for the factors that most impact churn at the company.

This notebook will prepare the data for analysis.

## Summary of Process
1. **Load dataset**: The dataset was loaded after downloading from Kaggle
2. **Compare column names to descriptions**: The column names were reviewed to see if they matched the descriptions and values. They appeared to be accurate.
3. **Review data types**: A review of the data types indicated that the TotalCharges column needs to be changed to a numeric data type.
    - **ISSUE**: After reviewing the column, it became clear that users in their first month of service (i.e., tenure = 0), had whitespace in the TotalCharges column. Additionally, given that those users with tenure = 1 had MonthlyCharges = TotalCharges, the whitespace for users in their first month of service was replaced with zero. Then the TotalCharges column was changed to numeric.
4. **Summary statistics & histograms**: The pandas profiler was used to generate summary statistics and histograms for each column and the dataset was summarized below.
5. **Outlier check**: The product of the tenure and monthly charges was calculated to see if the max values for total charges seem reasonable. While the max tenure x the max monthly charge was less than the max total charge, they were close enough to not seem like outliers. The histograms of each non-categorical column confirm that there are no outliers.
6. **Duplicate rows**: A check was done for duplicate rows but none were found.
7. **Drop customerID**: Given that the customerID column is not necessary for further analysis or modeling, it was dropped.

## Summary of Dataset
**Headline**: The dataset includes 7,043 distinct rows and no duplicates. **Most users do not churn: 74.5%.**

**Demographics**: The dataset contains roughly the same number of male (50.5%) and female (49.5%) users. Most users are not classified as senior citizens (84%) and roughly half (52%) do not have a partner. 

**Tenure**: The average tenure is 32.4 months with a median tenure of 29 months. Tenure ranges from 0 to 72 months, with 73 unique tenures. Tenure of 1, 2, 3, and 4 months are in the top five most common; 72 months is the second most common at 5.1% (1 month is the most common at 8.7%).

**Phone Service**: The vast majority of users have phone service (90%) and about half (48%) have a single phone line (42% have multiple lines).

**Internet Service**: Fewer users have internet service (78%) than phone service; 44% have fiber optic internet and 34% have DSL. 

**Internet-related services**: Most users with internet services do not have the various internet-related services:
    - 50% don't have security; 29% do
    - 44% don't have online backup; 35% do
    - 44% don't have device protection; 34% do
    - 49% don't have tech support; 29% do
    
**Streaming Services**: About half of users with internet (40% of total) have streaming TV and streaming movies.

**Contracts**: Most users have a month-to-month contract (55%). About a quarter (24%) have a 2-year contract and the remainder (21%) have a 1-year contract.

**Payment methods**: Most users opt for paperless billing (59%) and prefer to pay with electronic check (34%). Mailed checks, automatic bank transfers, and automatic credit card payments are all relatively similar at 21%-23%.

**Monthly Charges**: Monthly charges average \\$64.76 with a median of \\$70.35. Monthly charges range from \\$18.25 to \\$118.75 and there are 1585 distinct monthly charges in the dataset. The histogram appears to peak around \\$20/month.

**Total Charges**:Total charges average \\$2,279.73 with a median of \\$1,394.55. Total charges range from $0 (tenure = 0) to $8684.80. 75 percent of the users have a total charge of below $3,800.00.

## Data Sources
- WA_Fn-UseC_-Telco-Customer-Churn: [Kaggle Churn data set](https://www.kaggle.com/blastchar/telco-customer-churn/version/1)

## Changes
- 10-12-2021: Started project

## Import Libraries

In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np
from pandas_profiling import ProfileReport

## File Locations

In [2]:
today = datetime.today()
churn_file = Path.cwd() / "data" / "raw" / "WA_Fn-UseC_-Telco-Customer-Churn.csv"
summary_file = Path.cwd() / "data" / "processed" / f"summary.csv"

In [3]:
df = pd.read_csv(churn_file)

In [4]:
df['MonthlyCharges'].value_counts().sort_values()

78.70     1
62.55     1
40.90     1
26.50     1
56.00     1
         ..
20.00    43
19.90    44
19.95    44
19.85    45
20.05    61
Name: MonthlyCharges, Length: 1585, dtype: int64

## Data Definition and Cleaning
1. Identify all the column names and data types to see if they make sense for the description
2. Convert any data types as necessary
3. Change the SeniorCitizen values to Yes,No to match the other categorical variables
4. Use df.describe() and the pandas profilers to see statistics, unique values, and histograms for outliers
5. Check for non-categorical outliers with a product check for total charges (tenure * monthly charges)
6. Check for duplicate rows


### Identify all column names and data types
There are 21 columns in the dataset:

| Column Name | Description | Unique Values | Data Type |
| :--- | :--- | :--- | :---: |
| customerID | Unique identifer for each customer | N/A | object |
| gender | Whether the customer is a male or a female | Male, Female | object |
| SeniorCitizen | Whether the customer is a senior citizen or not | 1, 0 | integer |
| Partner | Whether the customer has a partner or not | Yes, No | object |
| Dependents | Whether the customer has dependents or not | Yes, No | object |
| tenure | Number of months the customer has stayed with the company | N/A | integer |
| PhoneService | Whether the customer has a phone service or not | Yes, No | object |
| MultipleLines | Whether the customer has multiple lines or not | Yes, No, No phone service | object |
| InternetService | Customer’s internet service provider | DSL, Fiber optic, No | object |
| OnlineSecurity | Whether the customer has online security or not | Yes, No, No internet service | object |
| OnlineBackup | Whether the customer has online backup or not | Yes, No, No internet service | object |
| DeviceProtection | Whether the customer has device protection or not | Yes, No, No internet service | object |
| TechSupport | Whether the customer has tech support or not | Yes, No, No internet service | object |
| StreamingTV | Whether the customer has streaming TV or not | Yes, No, No internet service | object |
| StreamingMovies | Whether the customer has streaming movies or not | Yes, No, No internet service | object |
| Contract | The contract term of the customer | Month-to-month, One year, Two year | object |
| PaperlessBilling | Whether the customer has paperless billing or not | Yes, No | object |
| PaymentMethod | The customer’s payment method | Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic) | object |
| MonthlyCharges | The amount charged to the customer monthly | N/A | float |
| TotalCharges | The total amount charged to the customer | N/A | float |
| Churn | Whether the customer churned or not | Yes, No | object |

**Note that TotalCharges originally had an object data type**

### Convert TotalCharges to numeric

In [5]:
# Isolate the rows that cause an error when attempting to convert TotalCharges to numeric
error_rows = df[pd.to_numeric(df['TotalCharges'], errors='coerce').isnull()]
error_rows

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


In [6]:
# Confirm the rows that have zero tenure
zero_tenure = df[df['tenure'] == 0]
zero_tenure

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


In [7]:
# Compare the zero tenure df to the error rows
set(error_rows) - set(zero_tenure)

set()

In [8]:
# Check to see what the TotalCharges is for the users with one month of tenure
one_tenure = df[df['tenure'] == 1]
one_tenure

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
20,8779-QRDMV,Male,1,No,No,1,No,No phone service,DSL,No,...,Yes,No,No,Yes,Month-to-month,Yes,Electronic check,39.65,39.65,Yes
22,1066-JKSGK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes
27,8665-UTDHZ,Male,0,Yes,Yes,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,30.20,30.2,Yes
33,7310-EGVHZ,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Bank transfer (automatic),20.20,20.2,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6979,5351-QESIO,Male,0,No,Yes,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,No,Mailed check,24.20,24.2,No
7010,0723-DRCLG,Female,1,Yes,No,1,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,74.45,74.45,Yes
7016,1471-GIQKQ,Female,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,49.95,49.95,No
7018,1122-JWTJW,Male,0,Yes,Yes,1,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,70.65,70.65,Yes


In [9]:
# Set the TotalCharges to zero for users with zero tenture (TotalCharges = MonthlyCharges for tenure = 1)
df.loc[df['tenure'] == 0, 'TotalCharges'] = 0

In [10]:
# Check the tenure = 0 rows
df[df['tenure'] == 0]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,0,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,0,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,0,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,0,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,0,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,0,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,0,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,0,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,0,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,0,No


In [11]:
# Convert TotalCharges to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [12]:
# Confirm the change
df['TotalCharges'].dtype

dtype('float64')

### Change the SeniorCitizen options to Yes, No from 1,0

In [13]:
df['SeniorCitizen'] = df['SeniorCitizen'].replace({0: 'No', 1: 'Yes'})

### Run summary statistics and profiling report

In [14]:
df.describe()

,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,2279.734304
std,24.559481,30.090047,2266.794470
min,0.000000,18.250000,0.000000
25%,9.000000,35.500000,398.550000
50%,29.000000,70.350000,1394.550000
75%,55.000000,89.850000,3786.600000
max,72.000000,118.750000,8684.800000


In [15]:
#profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
#profile

### Check for non-categorical outliers

In [16]:
summary = df.describe()
summary

,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,2279.734304
std,24.559481,30.090047,2266.794470
min,0.000000,18.250000,0.000000
25%,9.000000,35.500000,398.550000
50%,29.000000,70.350000,1394.550000
75%,55.000000,89.850000,3786.600000
max,72.000000,118.750000,8684.800000


In [17]:
summary['TotalCheck'] = (summary['tenure']*summary['MonthlyCharges']).round()

In [18]:
summary

,tenure,MonthlyCharges,TotalCharges,TotalCheck
count,7043.000000,7043.000000,7043.000000,49603849.0
mean,32.371149,64.761692,2279.734304,2096.0
std,24.559481,30.090047,2266.794470,739.0
min,0.000000,18.250000,0.000000,0.0
25%,9.000000,35.500000,398.550000,320.0
50%,29.000000,70.350000,1394.550000,2040.0
75%,55.000000,89.850000,3786.600000,4942.0
max,72.000000,118.750000,8684.800000,8550.0


### Check for duplicate rows

In [19]:
# Select duplicate rows except first occurrence based on all columns (adapted from: https://thispointer.com/pandas-find-duplicate-rows-in-a-dataframe-based-on-all-or-selected-columns-using-dataframe-duplicated-in-python/)
duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


### Drop customerID column

In [20]:
df = df.drop(['customerID'], axis=1)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,No,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [21]:
df['TotalCharges'].value_counts()

0.00       11
20.20      11
19.75       9
20.05       8
19.90       8
           ..
6849.40     1
692.35      1
130.15      1
3211.90     1
6844.50     1
Name: TotalCharges, Length: 6531, dtype: int64

## Save Output File

In [22]:
df.to_csv(summary_file, index=False)